## Pymongo CRUD
'''
bson id used objectid
'''

In [1]:
import pymongo
from bson import ObjectId

In [2]:
connection = pymongo.MongoClient("localhost", 27017)
database   = connection['db']
collection = database['inventory']

In [3]:
delres = collection.delete_many({})
print(str(delres.deleted_count)+' documents deleted')

0 documents deleted


In [4]:
document     = {'item': "canvas", 'qty': 100, 'tags': ["cotton"], 
                'size': { 'h': 28, 'w': 35.5, 'uom': "cm" }}
expl_id      = {'_id' : '00000123456789abcdefffff',
                'item': "journal", 'qty': 25, 'tags': ["blank", "red"], 
                'size': { 'h': 14, 'w': 21, 'uom': "cm" }}

### When single document is added then id will be automatically generated

In [5]:
insertonres = collection.insert_one(document)
auto_gen_id = insertonres.inserted_id
print(str(insertonres.inserted_id )+' is autogenerated Id')

5c649a100e21a5d59c9b1c5a is autogenerated Id


### Inserting using function -explicitly mentioned id

In [6]:
def insert_document(keyval):
    insertonres = collection.insert_one(keyval)
    return insertonres.inserted_id
docid = insert_document(expl_id)
print(str(docid)+' is explicited hex format id')

00000123456789abcdefffff is explicited hex format id


In [7]:
cursor = collection.insert_many([ { 'item': "mat", 'qty': 85, 'tags': ["gray"], 
                                                   'size': { 'h': 27.9, 'w': 35.5, 'uom': "cm" } },
                 { 'item': "mousepad", 'qty': 25, 'tags': ["gel", "blue"], 
                   'size': { 'h': 19, 'w': 22.85, 'uom': "cm" } }])
tuple(cursor.inserted_ids)

(ObjectId('5c649a100e21a5d59c9b1c5b'), ObjectId('5c649a100e21a5d59c9b1c5c'))

In [8]:
list([tuple(collection.find_one({'_id': ObjectId(str(auto_gen_id))}).items()),
tuple(collection.find({'item':'journal'})),
tuple(['Showing all documents -']),      
tuple(collection.find({}))])

[(('qty', 100),
  ('_id', ObjectId('5c649a100e21a5d59c9b1c5a')),
  ('tags', ['cotton']),
  ('size', {'h': 28, 'uom': 'cm', 'w': 35.5}),
  ('item', 'canvas')),
 ({'_id': '00000123456789abcdefffff',
   'item': 'journal',
   'qty': 25,
   'size': {'h': 14, 'uom': 'cm', 'w': 21},
   'tags': ['blank', 'red']},),
 ('Showing all documents -',),
 ({'_id': ObjectId('5c649a100e21a5d59c9b1c5a'),
   'item': 'canvas',
   'qty': 100,
   'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
   'tags': ['cotton']},
  {'_id': '00000123456789abcdefffff',
   'item': 'journal',
   'qty': 25,
   'size': {'h': 14, 'uom': 'cm', 'w': 21},
   'tags': ['blank', 'red']},
  {'_id': ObjectId('5c649a100e21a5d59c9b1c5b'),
   'item': 'mat',
   'qty': 85,
   'size': {'h': 27.9, 'uom': 'cm', 'w': 35.5},
   'tags': ['gray']},
  {'_id': ObjectId('5c649a100e21a5d59c9b1c5c'),
   'item': 'mousepad',
   'qty': 25,
   'size': {'h': 19, 'uom': 'cm', 'w': 22.85},
   'tags': ['gel', 'blue']})]

In [9]:
keyval = {'item': "journal", 'newkey': 'newvalue'} 
list([tuple(collection.find({'item':'journal'})),
#cursor = collection.update_one({'_id': ObjectId(auto_gen_id)}, {"$set": keyval}, upsert=True),
collection.update_one({'_id': ObjectId(auto_gen_id)}, {"$set": keyval}, upsert=True),
tuple(collection.find({'item':'journal'}))])

[({'_id': '00000123456789abcdefffff',
   'item': 'journal',
   'qty': 25,
   'size': {'h': 14, 'uom': 'cm', 'w': 21},
   'tags': ['blank', 'red']},),
 ({'_id': ObjectId('5c649a100e21a5d59c9b1c5a'),
   'item': 'journal',
   'newkey': 'newvalue',
   'qty': 100,
   'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
   'tags': ['cotton']},
  {'_id': '00000123456789abcdefffff',
   'item': 'journal',
   'qty': 25,
   'size': {'h': 14, 'uom': 'cm', 'w': 21},
   'tags': ['blank', 'red']})]

In [10]:
list([tuple(collection.find({'item':'journal'})),
      collection.update_many({'item':'journal'},{"$set": {'item' :'notebook',
                                                          'currentDate': {"lastModified": True}}}),
      tuple(collection.find({'item':'notebook'}))])

[({'_id': ObjectId('5c649a100e21a5d59c9b1c5a'),
   'item': 'journal',
   'newkey': 'newvalue',
   'qty': 100,
   'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
   'tags': ['cotton']},
  {'_id': '00000123456789abcdefffff',
   'item': 'journal',
   'qty': 25,
   'size': {'h': 14, 'uom': 'cm', 'w': 21},
   'tags': ['blank', 'red']}),
 ({'_id': ObjectId('5c649a100e21a5d59c9b1c5a'),
   'currentDate': {'lastModified': True},
   'item': 'notebook',
   'newkey': 'newvalue',
   'qty': 100,
   'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
   'tags': ['cotton']},
  {'_id': '00000123456789abcdefffff',
   'currentDate': {'lastModified': True},
   'item': 'notebook',
   'qty': 25,
   'size': {'h': 14, 'uom': 'cm', 'w': 21},
   'tags': ['blank', 'red']})]

In [11]:
list([tuple(collection.find({'item':'mousepad'})),
      collection.replace_one({'item': 'mousepad'},{'item': 'imac mousepad'}),
      tuple(collection.find({'item':'imac mousepad'}))
      ])

[({'_id': ObjectId('5c649a100e21a5d59c9b1c5c'),
   'item': 'mousepad',
   'qty': 25,
   'size': {'h': 19, 'uom': 'cm', 'w': 22.85},
   'tags': ['gel', 'blue']},),
 ({'_id': ObjectId('5c649a100e21a5d59c9b1c5c'), 'item': 'imac mousepad'},)]

In [12]:
def remove_data(keyval):
    delonresult = collection.delete_one(keyval)
    return delonresult.acknowledged
list([tuple(collection.find({'item':'imac mousepad'})),
      remove_data({'item':'imac mousepad'}),
      tuple(collection.find({'item':'imac mousepad'}))])

[({'_id': ObjectId('5c649a100e21a5d59c9b1c5c'), 'item': 'imac mousepad'},),
 True,
 ()]

### In 2 documents item values changed to notebook

In [13]:
list(collection.find({}))

[{'_id': ObjectId('5c649a100e21a5d59c9b1c5a'),
  'currentDate': {'lastModified': True},
  'item': 'notebook',
  'newkey': 'newvalue',
  'qty': 100,
  'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
  'tags': ['cotton']},
 {'_id': '00000123456789abcdefffff',
  'currentDate': {'lastModified': True},
  'item': 'notebook',
  'qty': 25,
  'size': {'h': 14, 'uom': 'cm', 'w': 21},
  'tags': ['blank', 'red']},
 {'_id': ObjectId('5c649a100e21a5d59c9b1c5b'),
  'item': 'mat',
  'qty': 85,
  'size': {'h': 27.9, 'uom': 'cm', 'w': 35.5},
  'tags': ['gray']}]

In [14]:
delres = collection.delete_many({})
print(str(delres.deleted_count)+' documents deleted')

3 documents deleted


In [15]:
list(collection.find({}))

[]

In [16]:
collection.drop()